# RegEx Workbook

In [46]:
# import modules
import json
import pandas as pd
import re
import regex
from datetime import datetime
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

In [47]:
# import twitter data
data_json = open('data/tweets/2017-12-01.json', mode='r').read()  
df = pd.read_json('data/tweets/2017-12-01.json')
df

,id,screen_name,user_id,time,link,text,source
0,936472671302414336,RoKhanna,771152516,2017-12-01T00:50:58-05:00,https://www.twitter.com/RoKhanna/statuses/9364...,Central problem for America’s economy. Stock m...,Twitter for iPhone
1,936470481011707904,jaredpolis,15361570,2017-12-01T00:42:16-05:00,https://www.twitter.com/jaredpolis/statuses/93...,If @JosephEStiglitz is so sure #bitcoin is a b...,Twitter for iPhone
2,936467554813521920,tedlieu,21059255,2017-12-01T00:30:38-05:00,https://www.twitter.com/tedlieu/statuses/93646...,@TheViKingDebate @TheJusticeDept Help us take ...,Twitter for Android
3,936466979573190656,RepAlGreen,156333623,2017-12-01T00:28:21-05:00,https://www.twitter.com/RepAlGreen/statuses/93...,"RT @RepAlGreen Next week, there will be a vote...",Twitter for iPhone
4,936465730538819584,tedlieu,21059255,2017-12-01T00:23:23-05:00,https://www.twitter.com/tedlieu/statuses/93646...,Dear @TheJusticeDept Sessions: If you have age...,Twitter for Android
...,...,...,...,...,...,...,...
3089,936808302633033728,RepDwightEvans,90639372,2017-12-01T23:04:39-05:00,https://www.twitter.com/nytimes/statuses/93680...,RT @nytimes Republicans are on the verge of pa...,Twitter for iPhone
3090,936808273109323776,RepDwightEvans,90639372,2017-12-01T23:04:32-05:00,https://www.twitter.com/SenWarren/statuses/936...,RT @SenWarren Well there you have it: The @Sen...,Twitter for iPhone
3091,936807981210988544,RepAdams,2916086925,2017-12-01T23:03:22-05:00,https://www.twitter.com/KamalaHarris/statuses/...,RT @KamalaHarris Senate Republicans are trying...,Twitter for iPhone
3092,936807771512565760,RepAdams,2916086925,2017-12-01T23:02:32-05:00,https://www.twitter.com/SenSchumer/statuses/93...,"RT @SenSchumer In my long career in politics, ...",Twitter for iPhone


In [48]:
df.dtypes

id              int64
screen_name    object
user_id         int64
time           object
link           object
text           object
source         object
dtype: object

# Data Preprocessing

In [49]:
# extract all hashtags into new column
df['hashtags'] = df.text.apply(lambda x: re.findall(r'#[\S]+', str(x)))
df.hashtags.head()

0                          []
1                  [#bitcoin]
2                          []
3    [#RepealandReplaceTrump]
4                          []
Name: hashtags, dtype: object

In [50]:
df.text[4]

'Dear @TheJusticeDept Sessions: If you have agents &amp; prosecutors with nothing to do, here are crimes more important than the stupid federal law against marijuana:\n\nChild pornography \nKidnapping \nBank Robbery \nIdentity Theft \n\nAnd are you ready for this ... here it comes\n\nPERJURY https://twitter.com/intelligencer/status/936347100501151744 QT @intelligencer Jeff Sessions on Wednesday suggested that the federal government is preparing to crack down on recreational marijuana http://nym.ag/2BxCAtQ'

In [51]:
df.text[6]

'RT @StephenAtHome Non-hypothetical question: are you insane? https://twitter.com/DonaldJTrumpJr/status/144505633981468672 QT @DonaldJTrumpJr Hypothetical question? Can my 2 year old get in trouble 4 sexual harassment for asking his teacher to come to his place naked after school?'

In [52]:
# extract all callouts
df['callouts'] = df.text.apply(lambda x: re.findall(r'@[\S]+', str(x)))
df['callouts'].head()

0                                     []
1                     [@JosephEStiglitz]
2    [@TheViKingDebate, @TheJusticeDept]
3                          [@RepAlGreen]
4      [@TheJusticeDept, @intelligencer]
Name: callouts, dtype: object

In [53]:
charref = [x for x in df.text.apply(lambda x: re.findall(r'&[\S]+', str(x)))]
charref
flat_list = [item for sublist in charref for item in sublist]
len(flat_list)

582

In [54]:
# replace all hashtags, ampersands, and callouts with no space
df.text = df.text.apply(lambda x: re.sub(r'[#@&][\S]+', '', str(x)))
df.text.head()

0    Central problem for America’s economy. Stock m...
1    If  is so sure  is a bubble, he should make a ...
2                         Help us take back the House.
3    RT  Next week, there will be a vote to impeach...
4    Dear  Sessions: If you have agents  prosecutor...
Name: text, dtype: object

In [55]:
# remove twitter RT and QT
df.text = df.text.apply(lambda x: re.sub(r'(\bRT\b|\bQT\b)', '', str(x)))
df.text.head()

0    Central problem for America’s economy. Stock m...
1    If  is so sure  is a bubble, he should make a ...
2                         Help us take back the House.
3      Next week, there will be a vote to impeach t...
4    Dear  Sessions: If you have agents  prosecutor...
Name: text, dtype: object

In [56]:
# remove emoji
df.text = df.text.apply(lambda x: re.sub(r'\B[^\w\s]{2,}\B', ' ', str(x)))
df.text.head()

0    Central problem for America’s economy. Stock m...
1    If  is so sure  is a bubble, he should make a ...
2                         Help us take back the House.
3      Next week, there will be a vote to impeach t...
4    Dear  Sessions: If you have agents  prosecutor...
Name: text, dtype: object

In [58]:
# remove html links
df.text = df.text.apply(lambda x:re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', str(x)))
df.text.head()

0    Central problem for America’s economy. Stock m...
1    If  is so sure  is a bubble, he should make a ...
2                         Help us take back the House.
3      Next week, there will be a vote to impeach t...
4    Dear  Sessions: If you have agents  prosecutor...
Name: text, dtype: object

In [59]:
# remove punctuation
df.text = df.text.apply(lambda x: re.sub(r'[^\w\s]', r'', str(x)))
df.text.head()

0    Central problem for Americas economy Stock mar...
1    If  is so sure  is a bubble he should make a f...
2                          Help us take back the House
3      Next week there will be a vote to impeach th...
4    Dear  Sessions If you have agents  prosecutors...
Name: text, dtype: object

In [60]:
# lower casing
df.text = df.text.apply(lambda x: " ".join(x.lower() for x in x.split()))
df.text.head()

0    central problem for americas economy stock mar...
1    if is so sure is a bubble he should make a for...
2                          help us take back the house
3    next week there will be a vote to impeach the ...
4    dear sessions if you have agents prosecutors w...
Name: text, dtype: object

In [61]:
# remove numbers or words with digits  
df.text = df.text.apply(lambda x: re.sub(r'\w*\d\w*', r'', str(x)))
df.text.head()

0    central problem for americas economy stock mar...
1    if is so sure is a bubble he should make a for...
2                          help us take back the house
3    next week there will be a vote to impeach the ...
4    dear sessions if you have agents prosecutors w...
Name: text, dtype: object

In [62]:
# remove any double or more whitespaces to single whitespaces
df.text = df.text.apply(lambda x: re.sub(r'\s\s+', ' ', str(x)))
df.text.head()

0    central problem for americas economy stock mar...
1    if is so sure is a bubble he should make a for...
2                          help us take back the house
3    next week there will be a vote to impeach the ...
4    dear sessions if you have agents prosecutors w...
Name: text, dtype: object

In [63]:
# remove any leading and trailing whitespace
df.text = df.text.apply(lambda x: re.sub(r'(\A\s+|\s+\Z)', '', str(x)))
df.text.head()

0    central problem for americas economy stock mar...
1    if is so sure is a bubble he should make a for...
2                          help us take back the house
3    next week there will be a vote to impeach the ...
4    dear sessions if you have agents prosecutors w...
Name: text, dtype: object

In [64]:
df.text.tail()

3089    republicans are on the verge of passing tax bi...
3090    well there you have it the wont even give demo...
3091    senate republicans are trying to force a vote ...
3092    in my long career in politics i have not seen ...
3093    nearly million hiv infections have been averte...
Name: text, dtype: object

# Time Column

In [44]:
df.time.head()

0    2017-12-01T00:50:58-05:00
1    2017-12-01T00:42:16-05:00
2    2017-12-01T00:30:38-05:00
3    2017-12-01T00:28:21-05:00
4    2017-12-01T00:23:23-05:00
Name: time, dtype: object

In [21]:
import regex
df['tdmatch'] = [x for x in df.time.apply(lambda x: regex.match(r'^(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2})T(?P<time>\d{2}:\d{2}:\d{2})-\d{2}:\d{2}$', str(x)))]
df.tdmatch = df.tdmatch.apply(lambda x: x.groupdict())
df.tdmatch.head()

0    {'year': '2017', 'month': '12', 'day': '01', '...
1    {'year': '2017', 'month': '12', 'day': '01', '...
2    {'year': '2017', 'month': '12', 'day': '01', '...
3    {'year': '2017', 'month': '12', 'day': '01', '...
4    {'year': '2017', 'month': '12', 'day': '01', '...
Name: tdmatch, dtype: object

In [22]:
df['year'] = df.tdmatch.apply(lambda x: x['year'])
df['month'] = df.tdmatch.apply(lambda x: x['month'])
df['time_hms'] = df.tdmatch.apply(lambda x: x['time'])
df['day'] = df.tdmatch.apply(lambda x: x['day'])

In [23]:
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 20)

In [24]:
df.head()

,id,screen_name,user_id,time,link,text,source,hashtags,callouts,html,tdmatch,year,month,time_hms,day
0,936472671302414336,RoKhanna,771152516,2017-12-01T00:50:58-05:00,https://www.twitter.com/RoKhanna/statuses/9364...,central problem americas economy stock market ...,Twitter for iPhone,[],[],[],"{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:50:58,01
1,936470481011707904,jaredpolis,15361570,2017-12-01T00:42:16-05:00,https://www.twitter.com/jaredpolis/statuses/93...,sure bubble should make fortune shorting matte...,Twitter for iPhone,[#bitcoin],[@JosephEStiglitz],"[(https, www.coindesk.com, /bitcoin-outlawed-e...","{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:42:16,01
2,936467554813521920,tedlieu,21059255,2017-12-01T00:30:38-05:00,https://www.twitter.com/tedlieu/statuses/93646...,help take back house,Twitter for Android,[],"[@TheViKingDebate, @TheJusticeDept]",[],"{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:30:38,01
3,936466979573190656,RepAlGreen,156333623,2017-12-01T00:28:21-05:00,https://www.twitter.com/RepAlGreen/statuses/93...,next week there will vote impeach bigotinchief...,Twitter for iPhone,[#RepealandReplaceTrump],[@RepAlGreen],"[(https, youtu.be, /ItfXf3e5Ozo)]","{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:28:21,01
4,936465730538819584,tedlieu,21059255,2017-12-01T00:23:23-05:00,https://www.twitter.com/tedlieu/statuses/93646...,dear sessions have agents prosecutors with not...,Twitter for Android,[],"[@TheJusticeDept, @intelligencer]","[(https, twitter.com, /intelligencer/status/93...","{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:23:23,01


In [25]:
format = "%Y-%m-%dT%H:%M:%S%z"
df['datetime'] = df.time.apply(lambda x: datetime.strptime(x, format))
df.datetime.head()

0   2017-12-01 00:50:58-05:00
1   2017-12-01 00:42:16-05:00
2   2017-12-01 00:30:38-05:00
3   2017-12-01 00:28:21-05:00
4   2017-12-01 00:23:23-05:00
Name: datetime, dtype: datetime64[ns, UTC-05:00]

In [26]:
df.head()

,id,screen_name,user_id,time,link,text,source,hashtags,callouts,html,tdmatch,year,month,time_hms,day,datetime
0,936472671302414336,RoKhanna,771152516,2017-12-01T00:50:58-05:00,https://www.twitter.com/RoKhanna/statuses/9364...,central problem americas economy stock market ...,Twitter for iPhone,[],[],[],"{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:50:58,01,2017-12-01 00:50:58-05:00
1,936470481011707904,jaredpolis,15361570,2017-12-01T00:42:16-05:00,https://www.twitter.com/jaredpolis/statuses/93...,sure bubble should make fortune shorting matte...,Twitter for iPhone,[#bitcoin],[@JosephEStiglitz],"[(https, www.coindesk.com, /bitcoin-outlawed-e...","{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:42:16,01,2017-12-01 00:42:16-05:00
2,936467554813521920,tedlieu,21059255,2017-12-01T00:30:38-05:00,https://www.twitter.com/tedlieu/statuses/93646...,help take back house,Twitter for Android,[],"[@TheViKingDebate, @TheJusticeDept]",[],"{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:30:38,01,2017-12-01 00:30:38-05:00
3,936466979573190656,RepAlGreen,156333623,2017-12-01T00:28:21-05:00,https://www.twitter.com/RepAlGreen/statuses/93...,next week there will vote impeach bigotinchief...,Twitter for iPhone,[#RepealandReplaceTrump],[@RepAlGreen],"[(https, youtu.be, /ItfXf3e5Ozo)]","{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:28:21,01,2017-12-01 00:28:21-05:00
4,936465730538819584,tedlieu,21059255,2017-12-01T00:23:23-05:00,https://www.twitter.com/tedlieu/statuses/93646...,dear sessions have agents prosecutors with not...,Twitter for Android,[],"[@TheJusticeDept, @intelligencer]","[(https, twitter.com, /intelligencer/status/93...","{'year': '2017', 'month': '12', 'day': '01', '...",2017,12,00:23:23,01,2017-12-01 00:23:23-05:00
